In [1]:
###### Set Up #####
# verify our folder with the data and module assets is installed
# if it is installed make sure it is the latest
!test -e ds-assets && cd ds-assets && git pull && cd ..
# if it is not installed clone it
!test ! -e ds-assets && git clone https://github.com/lutzhamel/ds-assets.git
# point to the folder with the assets
home = "ds-assets/assets/"
import sys
sys.path.append(home)      # add home folder to module search path

Already up to date.


# Classification Confidence Intervals

**Observation:** It does not matter how careful we are with our model evaluation techniques, there remains a fundamental uncertainty about the ability of our training data to effectively represent our (possibly infinite) data universe. This uncertainty can be observed during cross-validation: just partitioning the training data in different ways gives rise to drastic differences in model accuracy.  Here is the Iris example again from  the previous notebook.

In [2]:
# cross-validation Iris
import pandas as pd
import numpy as np
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn import tree
# grab cross validation code
from sklearn.model_selection import cross_val_score

# get data
df = pd.read_csv(home+"iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

# set up the model
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)

# do the 5-fold cross validation
scores = cross_val_score(model, X, y, cv=5)
print("Fold Accuracies: {}".format(scores))

Fold Accuracies: [0.93 0.97 0.90 0.87 1.00]


This uncertainty reflects into our model evaluation. If our training data is a poor representation of the data universe then the models we construct using it will generalize poorly to the rest of the data universe. If our training data is a good representation of the data universe then we can expect that our model will generalize well.

Here we will deal with this uncertainty using **confidence intervals.**
First, let us define confidence intervals formally. Given a model accuracy, *acc*, then the confidence interval is defined as the probability *p* that our model accuracy *acc* lies between some lower bound *lb* and some upper bound *ub*,

> $Pr(lb ≤ acc ≤ ub) = p.$

Paraphrasing this equation with *p = 95%*:

> We are 95% percent sure that our model accuracy is not worse than *lb* and not better than *ub*.


Ultimitely we are interested in the lower and upper bounds of the 95% confidence interval.  We can use the following formula to compute the bounds:

> $ub = acc + 1.96 \sqrt \frac{acc (1 - acc)}{n}$

> $lb = acc - 1.96 \sqrt \frac{acc (1 - acc)}{n}$

Here, *n* is the number of observations in the testing dataset used to estimate *acc*. The constant 1.96 is called the *z-score* and expresses the fact that we are computing the 95% confidence interval.


Let's do a simple example using the function `classification_confint`.

In [3]:
from confint import classification_confint

# assume the following
observations = 100
acc = .88

# then
lb,ub = classification_confint(acc,observations)
print('Accuracy with confidence interval: {} ({:3.2f},{:3.2f})'.format(acc,lb, ub))

Accuracy with confidence interval: 0.88 (0.82,0.94)


Now, let's do an actual example using the abalone dataset.  We want to print out the testing accuracy together with it's 95% confidence interval.

In [4]:
# Grid search with cross-validation for abalone dataset
import pandas as pd
from sklearn import tree
from confint import classification_confint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# get data
df = pd.read_csv(home+"abalone.csv")
X  = df.drop(columns=['sex'])
y = df[['sex']]

# setting up grid search
model = tree.DecisionTreeClassifier(random_state=1)
param_grid = {
    'max_depth': list(range(1,11)), # search 1..10
    'criterion': ['entropy', 'gini', 'log_loss']
    }
grid = GridSearchCV(model, param_grid, cv=5)

# performing grid search
grid.fit(X,y)

# compute the accuracy of optimal classifier
predict_y = grid.best_estimator_.predict(X)
acc = accuracy_score(y, predict_y)
observations = X.shape[0]
lb,ub = classification_confint(acc,observations)

# print accuracy
print("Accuracy of optimal classifier with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy of optimal classifier with confidence interval: 0.59 (0.57, 0.60)


# Regression Confidence Intervals

When performing regression we use the $R^2$ score to examine the quality of our models.  Given that we only use a small training dataset for fitting the model compared to the rest of the data universe it is only natural to ask what the 95% confidence interval for this score might be.  We have a formula for that -- it is not as straight forward as the confidence interval for classification,

> $lb = R^2 - 2\sqrt{\frac{4R^{2}(1-R^{2})^{2}(n-k-1)^{2}}{(n^2 - 1)(n+3)}}$

> $ub = R^2 + 2\sqrt{\frac{4R^{2}(1-R^{2})^{2}(n-k-1)^{2}}{(n^2 - 1)(n+3)}}$

Here, *n* is the number of observations in the validation/testing dataset and *k* is the number of independent variables.

In [5]:
from confint import regression_confint

# assume the following
rs_score = .75
observations = 100
variables = 4 # independent variables

# then
lb,ub = regression_confint(rs_score, observations, variables)
print("R^2 Score with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(rs_score,lb,ub))

R^2 Score with confidence interval: 0.75 (0.67, 0.83)


Let's look at an actual regression problem and compute the $R^2$ score and it's 95% confidence interval. We will use the cars problem from before.

In [6]:
import numpy as np
import pandas
from sklearn.tree import DecisionTreeRegressor
from confint import regression_confint

# get our dataset
cars_df = pandas.read_csv(home+"cars.csv")

# build model object
model = DecisionTreeRegressor(max_depth=None)

# fit model
# We have to reshape the values array to make 'fit' happy because
# the array only has a single feature
model.fit(cars_df[['speed']],cars_df[['dist']])

# R^2 score
rs_score = model.score(cars_df[['speed']],cars_df[['dist']])
observations = cars_df.shape[0]
variables = 1
lb,ub = regression_confint(rs_score, observations, variables)

# print out R^2 score with its 95% confidence interval
print("R^2 Score with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(rs_score,lb,ub))

R^2 Score with confidence interval: 0.79 (0.69, 0.89)


# Statistical Significance

Besides giving us an idea of the uncertainty of our model the 95% confidence intervals also have something to say about the significance of scores of different models.  That is, if the confidence intervals overlap then the difference in model performance of two different models on the same dataset is not statistically significant.

Consider the following,

In [7]:
from confint import classification_confint

observations = 100

# assume first classifier
acc1 = .88
lb1,ub1 = classification_confint(acc1,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc1,lb1, ub1))

# assume second classifier
acc2 = .92
lb2,ub2 = classification_confint(acc2,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc2,lb2, ub2))

Accuracy: 0.88 (0.82,0.94)
Accuracy: 0.92 (0.87,0.97)


Even though the second classifier has a better raw accuracy when we look at the confidence intervals of the two classifiers we see that they overlap.  Here we see that the first classifier could potentially have an accuracy of .94 (even better than the raw accuracy of the second classifier).  Furthermore, the confidence interval of the second classifier tells us that that classifier could potentially have an accuracy of .87 which is worse than the raw accuracy of the first classifier.  For this reason we say that the difference in accuracy of two classifiers is not statistically significant if their confidence intervals overlap.

## A Worked Example

First we will construct the optimal model for the abalone data set which is the same code as we have seen above and then we construct a minimal tree for the same data set and compare the performances using statistical significance.

The optimal tree first.

In [8]:
# Grid search with cross-validation for abalone dataset
import pandas as pd
from sklearn import tree
from confint import classification_confint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# get data
df = pd.read_csv(home+"abalone.csv")
X  = df.drop(columns=['sex'])
y = df[['sex']]

# setting up grid search
model = tree.DecisionTreeClassifier(random_state=1)
param_grid = {
    'max_depth': list(range(1,11)), # search 1..10
    'criterion': ['entropy', 'gini', 'log_loss']
    }
grid = GridSearchCV(model, param_grid, cv=5)

# performing grid search
grid.fit(X,y)

# compute the accuracy of optimal classifier
predict_y = grid.best_estimator_.predict(X)
acc = accuracy_score(y, predict_y)
observations = X.shape[0]
lb,ub = classification_confint(acc,observations)

# print accuracy
print("Accuracy of optimal classifier with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy of optimal classifier with confidence interval: 0.59 (0.57, 0.60)


Now we construct the minimal tree.

In [9]:
# create our model object
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=1)

# train
model.fit(X,y)

# compute the accuracy of minimal classifier
predict_y = model.predict(X)
acc = accuracy_score(y, predict_y)
observations = X.shape[0]
lb,ub = classification_confint(acc,observations)

# print accuracy
print("Accuracy of minimal classifier with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy of minimal classifier with confidence interval: 0.53 (0.52, 0.55)


**Observation**: The confidence intervals are not overlapping, therefore the performance difference is statistically significant!

# Project

Please see BrightSpace.